Creates 10 sealed pools, along with the communities generated within the pool by https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html.

Outputs results as HTML files in the `sealed_pools` directory in this directory.

In [ ]:
from mtg_draft_ai.api import *

cards = read_cube_toml('cube_81183_tag_data.toml')

In [ ]:
# Jupyter magic - run shell command
!mkdir sealed_pools

In [ ]:
def contains_all_colors(cards, colors):
    colors_seen = set()
    for card in cards:
        if card.color_id != 'C':
            for color in card.color_id:
                colors_seen.add(color)
    
    return colors_seen == set(colors)

In [ ]:
from networkx.algorithms import community
from mtg_draft_ai.display import cards_to_html
from mtg_draft_ai import synergy
import random

def write_pool(cards, filename):
    pool = cards.copy()
    random.shuffle(pool)
    pool = sorted(pool[:90], key=lambda card: card.color_id)

    graph = synergy.create_graph(cards)
    
    output_html = 'Pool:\n<br>\n{}<br>\n'.format(cards_to_html(pool))

    for colors in ['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG']:
        output_html += 'Colors: {} <br>\n'.format(colors)
        subgraph = synergy.colors_subgraph(graph.subgraph(pool), colors)
        colors_gmc = community.greedy_modularity_communities(subgraph)

        i = 0
        for gmc in colors_gmc:
            if not contains_all_colors(gmc, colors):
                continue

            gmc_subgraph = subgraph.subgraph(gmc)
            sorted_community_cards = [k for k, v in synergy.sorted_centralities(gmc_subgraph)]

            output_html += 'Community {}\n'.format(i)
            output_html += '<div>\n{}</div>\n'.format(cards_to_html(sorted_community_cards))
            i += 1

    with open(filename, 'w') as f:
        f.write(output_html)
#print(output_html)

In [ ]:
for i in range(1, 10):
    write_pool(cards, 'sealed_pools/sealed{}.html'.format(i))